# District catigorization on various demographic indicators
The goal of this project is to creata more accurate benchmarks for school district preformance. When comparing the preformance of school districts often state or neigboring school districts are used as benchmarks; even when socio-dconomic factors can be extreamly diffrent. Some notable factors that I think may cause districts to preform diffrently is teacher compensation,  teacher student ethnic distributions (speficcily ethnic homogeneity), 

as stronger homoginininty would corilate to stronger outcomes using chorpis christi and the RGV as an example where high student teacher ethnic hogoininity shows high student preformane and outcomes when looking at STAAR or college redinesss scores in hispanic communities leading to high student preformance.



In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

## Clean and create Master Districts Dataset

```district_master``` - This is a cleaned up list of school districts pulled together from 2013 & 2022-2025 years of data, keeping only the districts and fields that exist in all 5 years i.e. district ID, district name, county, region, and a couple of simple flags like whether it’s a charter so I can quickly see what each district.

***To-Dos / Notes***
- This needs a lot more cleaning...
- Re-title columns
- I will eventually need to re-clean this because rn It very bare bones
- I need to consider charter is not separated
- Double-check that matching district IDs actually represent the same real district across all years
- for all incoming data I will need to remove districts not in my district_final_cut 
- and if I go back more than 2022 if one of my districts in the final cut disappears then I need to remove them from the consideration to.
- 

i.e. 
a district is not listed for any number of reasons before 2019 but in my current list of districts from 2022-2025 if i used data back to 2014 then it will skew... actually I just loaded in 2012 and added that to the set cross refrence and it narrowd down the list a little more so im just going with those ones. 




In [2]:
# Load in District data
district_2025 = pd.read_csv(r'District Infromation\2025 District Reference.csv')
district_2024 = pd.read_csv(r'District Infromation\2024 District Reference.csv')
district_2023 = pd.read_csv(r'District Infromation\2023 District Reference Information.csv')
district_2022 = pd.read_csv(r'District Infromation\2022 District Reference Information, Accountability Rating and Special Education Determination Status.csv')
district_2013 = pd.read_csv(r'District Infromation/2013 District Reference .dat', dtype=str)

In [3]:
district_25_headers = district_2025.iloc[0]
district_25 = district_2025[1:]
district_25.columns = district_25_headers
district_25.insert(loc=0, column="Year", value="2025")

district_25.head(1)

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,DFLCHART,DFLALTED,D_RATING,DAD_POST,ASVAB_STATUS,OUTCOME
1,2025,001902,CAYUGA ISD,001,ANDERSON,07,REGION 07: KILGORE,N,N,B,1,,Meets Requirements


In [4]:
district_2024_headers = district_2024.iloc[0]
district_24 = district_2024[1:]
district_24.columns = district_2024_headers
district_24.insert(loc=0, column="Year", value="2024")

district_24.head(1)

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,REGNNAME,DFLCHART,DFLALTED,D_RATING,DAD_POST,ASVAB_STATUS,OUTCOME
1,2024,001902,CAYUGA ISD,001,ANDERSON,07,REGION 07: KILGORE,N,N,B,1,,Meets Requirements


In [5]:
district_2022

,COUNTY,REGION,DISTRICT,D_rating,DISTNAME,CNTYNAME,DFLCHART,DFLALTED,DAD_POST,OUTCOME,asvab_status
0,'001,'07,'001902,A,CAYUGA ISD,ANDERSON,N,N,1,Meets Requirements,
1,'001,'07,'001903,A,ELKHART ISD,ANDERSON,N,N,0,Meets Requirements,
2,'001,'07,'001904,A,FRANKSTON ISD,ANDERSON,N,N,0,Meets Requirements,ASVAB Alternative Test Offered
3,'001,'07,'001906,A,NECHES ISD,ANDERSON,N,N,1,Meets Requirements,
4,'001,'07,'001907,B,PALESTINE ISD,ANDERSON,N,N,0,Needs Assistance,
...,...,...,...,...,...,...,...,...,...,...,...
1202,'252,'09,'252902,A,NEWCASTLE ISD,YOUNG,N,N,,Meets Requirements,
1203,'252,'09,'252903,A,OLNEY ISD,YOUNG,N,N,0,Meets Requirements,
1204,'253,'01,'253901,B,ZAPATA COUNTY ISD,ZAPATA,N,N,0,Needs Assistance,
1205,'254,'20,'254901,B,CRYSTAL CITY ISD,ZAVALA,N,N,0,Needs Intervention,


In [6]:
district_2022

,COUNTY,REGION,DISTRICT,D_rating,DISTNAME,CNTYNAME,DFLCHART,DFLALTED,DAD_POST,OUTCOME,asvab_status
0,'001,'07,'001902,A,CAYUGA ISD,ANDERSON,N,N,1,Meets Requirements,
1,'001,'07,'001903,A,ELKHART ISD,ANDERSON,N,N,0,Meets Requirements,
2,'001,'07,'001904,A,FRANKSTON ISD,ANDERSON,N,N,0,Meets Requirements,ASVAB Alternative Test Offered
3,'001,'07,'001906,A,NECHES ISD,ANDERSON,N,N,1,Meets Requirements,
4,'001,'07,'001907,B,PALESTINE ISD,ANDERSON,N,N,0,Needs Assistance,
...,...,...,...,...,...,...,...,...,...,...,...
1202,'252,'09,'252902,A,NEWCASTLE ISD,YOUNG,N,N,,Meets Requirements,
1203,'252,'09,'252903,A,OLNEY ISD,YOUNG,N,N,0,Meets Requirements,
1204,'253,'01,'253901,B,ZAPATA COUNTY ISD,ZAPATA,N,N,0,Needs Assistance,
1205,'254,'20,'254901,B,CRYSTAL CITY ISD,ZAVALA,N,N,0,Needs Intervention,


In [ ]:
# Strip the leading ' in front of County, Region, District (most likely there due to excel being common for analysis and it would keep those as strings not ints)
district_2022['COUNTY'] = district_2022['COUNTY'].str[1:]
district_2022['REGION'] = district_2022['REGION'].str[1:]
district_2022['DISTRICT'] = district_2022['DISTRICT'].str[1:]

district_2023['COUNTY'] = district_2023['COUNTY'].str[1:]
district_2023['REGION'] = district_2023['REGION'].str[1:]
district_2023['DISTRICT'] = district_2023['DISTRICT'].str[1:]

In [ ]:
# Add years as a column
district_2013.insert(loc=0, column="Year", value="2013")
district_2022.insert(loc=0, column="Year", value="2022")
district_2023.insert(loc=0, column="Year", value="2023")

district_22 = district_2022
district_23 = district_2023

In [ ]:
# This is a cleaned list of school districts I’m keeping by taking only the districts that show up in every dataset (2025, 2024, 2023, 2022, and 2013), 
# based on the assumption that if a district ID exists across all years, it’s stable enough to trust and include.

# To-do: Double-check that matching district IDs actually represent the same real district across all years
d_25 = set(district_25['DISTRICT'])
d_24 = set(district_24['DISTRICT'])
d_23 = set(district_23['DISTRICT'])
d_22 = set(district_22['DISTRICT'])
d_12 = set(district_2013['DISTRICT'])

# Creating a set of districts only in all
District_final_cut = d_25 & d_24 & d_23 & d_22 & d_12
District_final_cut
len(District_final_cut)


1162

In [ ]:
# Check district lengths aren't changing  
# 2025: Original List Len: 1208 New List Len: 1208
# 2024: Original List Len: 1207 New List Len: 1207
# 2023: Original List Len: 1209 New List Len: 1209
# 2022: Original List Len: 1207 New List Len: 1207

print(f'Original List Len: {len(district_22['DISTRICT'])} New Set Len: {len(d_22)}')


Original List Len: 1207 New Set Len: 1207


In [12]:
def remove_districts(districts):
    filter = districts['DISTRICT'].isin(District_final_cut)
    filtered_district = districts[filter]
    return filtered_district

In [13]:
district_25_filtered = remove_districts(district_25)
district_24_filtered = remove_districts(district_24)
district_23_filtered = remove_districts(district_23)
district_22_filtered = remove_districts(district_22)
district_13_filtered = remove_districts(district_2013)


In [14]:
dist_25_col = set(district_25_filtered.columns)
dist_24_col = set(district_24_filtered.columns)
dist_23_col = set(district_23_filtered.columns)
dist_22_col = set(district_22_filtered.columns)
dist_13_col = set(district_2013.columns)

dist_col_in_all = dist_25_col & dist_24_col & dist_23_col & dist_22_col & dist_13_col
dist_col_in_all

{'CNTYNAME',
 'COUNTY',
 'DFLALTED',
 'DFLCHART',
 'DISTNAME',
 'DISTRICT',
 'REGION',
 'Year'}

In [15]:
# now I need to make a new data frame that is a version of all of these where all the data is the same, and I want to see what wrong with the ones that arent.
# i.e. if a cdn is around an but the district name is diff or county etc

dist_df_list = [district_25_filtered, district_24_filtered, district_23_filtered, district_22_filtered, district_13_filtered]

districts_grouped = pd.concat(
    [df.loc[:, df.columns.intersection(dist_col_in_all)] for df in dist_df_list]   
)

districts_grouped

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED
1,2025,001902,CAYUGA ISD,001,ANDERSON,07,N,N
2,2025,001903,ELKHART ISD,001,ANDERSON,07,N,N
3,2025,001904,FRANKSTON ISD,001,ANDERSON,07,N,N
4,2025,001906,NECHES ISD,001,ANDERSON,07,N,N
5,2025,001907,PALESTINE ISD,001,ANDERSON,07,N,N
...,...,...,...,...,...,...,...,...
1223,2013,252902,NEWCASTLE ISD,252,YOUNG,09,N,N
1224,2013,252903,OLNEY ISD,252,YOUNG,09,N,N
1225,2013,253901,ZAPATA COUNTY ISD,253,ZAPATA,01,N,N
1226,2013,254901,CRYSTAL CITY ISD,254,ZAVALA,20,N,N


In [16]:
dist_df_list

[0     Year DISTRICT           DISTNAME COUNTY  CNTYNAME REGION  \
 1     2025   001902         CAYUGA ISD    001  ANDERSON     07   
 2     2025   001903        ELKHART ISD    001  ANDERSON     07   
 3     2025   001904      FRANKSTON ISD    001  ANDERSON     07   
 4     2025   001906         NECHES ISD    001  ANDERSON     07   
 5     2025   001907      PALESTINE ISD    001  ANDERSON     07   
 ...    ...      ...                ...    ...       ...    ...   
 1204  2025   252902      NEWCASTLE ISD    252     YOUNG     09   
 1205  2025   252903          OLNEY ISD    252     YOUNG     09   
 1206  2025   253901  ZAPATA COUNTY ISD    253    ZAPATA     01   
 1207  2025   254901   CRYSTAL CITY ISD    254    ZAVALA     20   
 1208  2025   254902       LA PRYOR ISD    254    ZAVALA     20   
 
 0                     REGNNAME DFLCHART DFLALTED D_RATING DAD_POST  \
 1           REGION 07: KILGORE        N        N        B        1   
 2           REGION 07: KILGORE        N        N   

In [17]:
# Check how many unique 'charter_status' values each district has
charter_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

aea_check = districts_grouped.groupby('DISTRICT')['DFLALTED'].nunique()

# IDs where the status changed (more than 1 unique value)
# inconsistent_ids = charter_check[charter_check > 1].index  # Nothing
inconsistent_ids = aea_check[aea_check > 1].index
inconsistent_ids

Index(['015807', '015808', '101804', '101837', '101842', '101864', '193801'], dtype='str', name='DISTRICT')

In [18]:
districts_grouped.shape

districts_cleaned = districts_grouped[~districts_grouped['DISTRICT'].isin(inconsistent_ids)]

print(districts_grouped.shape, districts_cleaned.shape)


(5810, 8) (5775, 8)


In [19]:
districts_cleaned

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED
1,2025,001902,CAYUGA ISD,001,ANDERSON,07,N,N
2,2025,001903,ELKHART ISD,001,ANDERSON,07,N,N
3,2025,001904,FRANKSTON ISD,001,ANDERSON,07,N,N
4,2025,001906,NECHES ISD,001,ANDERSON,07,N,N
5,2025,001907,PALESTINE ISD,001,ANDERSON,07,N,N
...,...,...,...,...,...,...,...,...
1223,2013,252902,NEWCASTLE ISD,252,YOUNG,09,N,N
1224,2013,252903,OLNEY ISD,252,YOUNG,09,N,N
1225,2013,253901,ZAPATA COUNTY ISD,253,ZAPATA,01,N,N
1226,2013,254901,CRYSTAL CITY ISD,254,ZAVALA,20,N,N


In [20]:
# using 2025 as source of truth

district_master = districts_cleaned[districts_cleaned["Year"]=='2025'].reset_index(drop=True)


district_master.head()

,Year,DISTRICT,DISTNAME,COUNTY,CNTYNAME,REGION,DFLCHART,DFLALTED
0,2025,001902,CAYUGA ISD,001,ANDERSON,07,N,N
1,2025,001903,ELKHART ISD,001,ANDERSON,07,N,N
2,2025,001904,FRANKSTON ISD,001,ANDERSON,07,N,N
3,2025,001906,NECHES ISD,001,ANDERSON,07,N,N
4,2025,001907,PALESTINE ISD,001,ANDERSON,07,N,N


## Clean Student & Staff Datasets

```staff_master``` - This is all staff data from 2013 through 2025 merged the same way, with only the shared columns kept so everything lines up cleanly year to year.

```student_master``` - This is all student data from 2013 through 2025 merged together, keeping only the columns that exist in every year so the data is consistent across time.

***To-Dos / Notes***
- This needs a lot more cleaning...
- Account for any data masking (haven't handled that yet)
- Add testing and validation throughout the process to validate each step.
- Clean up the keys table, to account for columns that are the same thing but named differently depending on the yea, handle fields that only exist in some years but are still useful.

***Useful links***
- View had data is masked: https://rptsvr1.tea.texas.gov/perfreport/tapr/2025/masking.html
- 2013 data download headers explained: https://rptsvr1.tea.texas.gov/perfreport/tapr/2013/download/taprref.html

In [21]:
# Load in all staff data
StudentStaff_Info_2013 = pd.read_csv(r"Staff & Student Data\2013 Staff & Student Information.txt")
StudentStaff_Info_2014 = pd.read_csv(r"Staff & Student Data\2014 Staff & Student Information.dat")
StudentStaff_Info_2015 = pd.read_csv(r"Staff & Student Data\2015 Staff & Student Information.dat")
StudentStaff_Info_2016 = pd.read_csv(r"Staff & Student Data\2016 District Staff & Student Information.dat")
StudentStaff_Info_2017 = pd.read_csv(r"Staff & Student Data\2017 Staff & Student Information.dat")
StudentStaff_Info_2018 = pd.read_csv(r"Staff & Student Data\2018 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2019 = pd.read_csv(r"Staff & Student Data\2019 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2020 = pd.read_csv(r"Staff & Student Data\2020 Staff, Student, and Annual Graduates.dat")
StudentStaff_Info_2021 = pd.read_csv(r"Staff & Student Data\2021 Staff, Student, and Annual Graduates.csv")
StudentStaff_Info_2022 = pd.read_csv(r"Staff & Student Data\2022 Staff, Student, and Annual Graduates.csv")
StudentStaff_Info_2023 = pd.read_csv(r"Staff & Student Data\2023 Staff, Student, and Annual Graduates.csv")
Student_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Student Information.csv")
Staff_Info_2024 = pd.read_csv(r"Staff & Student Data\2024 District Staff Information.csv")
Student_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Student Information.csv")
Staff_Info_2025 = pd.read_csv(r"Staff & Student Data\2025 District Staff Information.csv")


In [22]:
# Explore data
StudentStaff_Info_2014.columns

Index(['DISTRICT', 'DA0GH13N', 'DA0GR13N', 'DA0GM13N', 'DB0GR13N', 'DH0GR13N',
       'DW0GR13N', 'DI0GR13N', 'D30GR13N', 'D40GR13N',
       ...
       'DPST01SA', 'DPST06SA', 'DPST11SA', 'DPST20SA', 'DPSTTOSA', 'DPSUTOSA',
       'DPSSTOSA', 'DPSCTOSA', 'DPFEIERP', 'DPSAINHP'],
      dtype='str', length=250)

### Seprate Student and Staff Data 2013 - 2023

- When I downloaded the datasets for student and staff information 2013 and 2023 the student staff data was combined. I will want to separate these later but to make managing this many tables scalable I will throw then in a dict. ```Note: I will add 2024 and 2025 separated student and staff DataFrames to a dictionary later.```
- Load in the student staff key files

Future Goal: Combining all student and staff data downloads in master keys based on all unique rows but i think a function will be quicker for me to do now so I'll merge the headers later

In [23]:
# Add student and staff 2013 and 2023 combined dataframes to a dictionary.

student_staff_dfs = {2013:StudentStaff_Info_2013, 
                      2014:StudentStaff_Info_2014, 
                      2015:StudentStaff_Info_2015, 
                      2016:StudentStaff_Info_2016, 
                      2017:StudentStaff_Info_2017,
                      2018:StudentStaff_Info_2018,
                      2019:StudentStaff_Info_2019, 
                      2020:StudentStaff_Info_2020,
                      2021:StudentStaff_Info_2021,
                      2022:StudentStaff_Info_2022,
                      2023:StudentStaff_Info_2023}
type(student_staff_dfs)

dict

In [24]:
# Loading in all the student and staff keys for 2013 to 2024
student_key_tables ={}
staff_key_tables = {}

for year in range(2013, 2024):
    # Load in files with a year variable
    student_key = pd.read_csv(rf"Staff & Student Keys\{year}_TAPR_Advanced_Data_Download_Student.csv")
    staff_key = pd.read_csv(rf"Staff & Student Keys\{year}_TAPR_Advanced_Data_Download_Staff.csv")

    student_key_tables[year] = student_key
    staff_key_tables[year] = staff_key

print(len(student_key_tables), len(staff_key_tables))


11 11


In [25]:
# For the 2024 and 2025 school years, data sets had the header a human readable name I stripped that for now and promoted index 0 which used the old id naming conversion

# Note: I'll eventually need to save those header
def PromoteFirstRow(df, year):  
    new_headers = df.iloc[0]
    new_df = df.iloc[1:]
    new_df.columns = new_headers
    new_df = new_df.reset_index(drop=True)
    key_df = pd.DataFrame(new_headers)
    key_df = key_df.reset_index()
    key_df.columns = ["LABEL", "NAME"]

    return new_df, key_df

Student_Info_2024, key_df_stu_24 = PromoteFirstRow(Student_Info_2024, 2024)
student_key_tables[2024] = key_df_stu_24

Student_Info_2025, key_df_stu_25= PromoteFirstRow(Student_Info_2025, 2025)
student_key_tables[2025] = key_df_stu_25

Staff_Info_2024, key_df_staff_24= PromoteFirstRow(Staff_Info_2024, 2024)
staff_key_tables[2024] = key_df_staff_24

Staff_Info_2025, key_df_staff_25= PromoteFirstRow(Staff_Info_2025, 2025)
staff_key_tables[2025] = key_df_staff_25


In [26]:
# Creating two dictionary one holding all the student data another holding all the staff data 

student_tables = {}
staff_tables = {}

for year in range(2013, 2024):
    # In the dictionary student_tables we have {year: DataFrame} i want to copy the data from the respective 


    student_tables[year] = student_staff_dfs[year][student_key_tables[year]['NAME']].copy()
    staff_tables[year] = student_staff_dfs[year][staff_key_tables[year]['NAME']].copy()

    student_tables[year].insert(loc=0, column="Year", value=year)
    staff_tables[year].insert(loc=0, column="Year", value=year)

    if year == 2023:

        
        student_tables[2024] = Student_Info_2024
        student_tables[2025] = Student_Info_2025
        student_tables[2024].insert(loc=0, column="Year", value=2024)
        student_tables[2025].insert(loc=0, column="Year", value=2025)

        staff_tables[2024] = Staff_Info_2024
        staff_tables[2025] = Staff_Info_2025
        staff_tables[2024].insert(loc=0, column="Year", value=2024)
        staff_tables[2025].insert(loc=0, column="Year", value=2025)



In [27]:
# doing a basic test to see if the shapes make sense not sure why its plus one but not worries enough to check rn... i feel like it something to do with counting a header or index
for i in range(2013, 2024):    
    print(f"[{i}]: New Shape: {staff_tables[i].shape}, Expected shape: ({student_staff_dfs[i].shape[0]}, {staff_key_tables[i].shape[0]})")

[2013]: New Shape: (1228, 138), Expected shape: (1228, 137)
[2014]: New Shape: (1227, 138), Expected shape: (1227, 137)
[2015]: New Shape: (1219, 138), Expected shape: (1219, 137)
[2016]: New Shape: (1207, 138), Expected shape: (1207, 137)
[2017]: New Shape: (1203, 151), Expected shape: (1203, 150)
[2018]: New Shape: (1200, 155), Expected shape: (1200, 154)
[2019]: New Shape: (1201, 155), Expected shape: (1201, 154)
[2020]: New Shape: (1202, 116), Expected shape: (1202, 115)
[2021]: New Shape: (1204, 121), Expected shape: (1204, 120)
[2022]: New Shape: (1207, 127), Expected shape: (1207, 126)
[2023]: New Shape: (1209, 121), Expected shape: (1209, 120)


#### Current Data State (Checkpoint)

As of now, the data has been manually cleaned and separated into two main structures:

- One dictionary containing **student dataframes**
- One dictionary containing **staff dataframes**

Each dictionary is keyed by **year**, covering **2013–2023**.

For both students and staff:
- Each dataframe represents a single year
- District is present and serves as the ID
- Year will be added or already exists as a column
- Data is mostly clean and usable for analysis

At this point:
- Student and staff data are fully separated
- No further loading or restructuring is needed

This is a stable checkpoint. Further refactoring or automation can be done later if needed.

In [28]:
# Make a set/list of all the headers that exist across all the years data. Think District, Year, or columns like DPETWHIC. Doing this so i can Grab those columns later
staff_columns_by_year = {
    year: set(df.columns)
    for year, df in staff_tables.items()
    }

student_columns_by_year = {
    year: set(df.columns)
    for year, df in student_tables.items()
    }

# columns that exist in ALL years
staff_common_columns = set.intersection(*staff_columns_by_year.values())
student_common_columns = set.intersection(*student_columns_by_year.values())


In [29]:
# Eventually I will need to clean the keys and add add any keys that either had small naming diff because of years or if I am ok with missing x numbers of years
# for something. like maybe they had chronic absenteeism on here at one point.

In [30]:
# Make a new dict of data frames but make these data frames only ones with matching columns

staff_matching_col_tables = {}
student_matching_col_tables = {}

for year in staff_tables.keys():
    staff_matching_col_tables[year] = staff_tables[year][list(staff_common_columns)]

for year in student_tables.keys():
    student_matching_col_tables[year] = student_tables[year][list(student_common_columns)]

In [31]:
staff_matching_col_tables[2025].head(2)

,DPSTOPFP,DPSECOFC,DPSSTOFC,DPSXTOFC,DPST06ST,DPSTASFC,DPSTBAFP,DPSTURNR,DPSTFEFC,DPSTPHFC,DPSTVOFC,DPSCTOSA,DPST01ST,DPSTGIFP,DPSPTOFC,DPSTBLFP,DPSUTOFC,DPSUTOFP,DPSTSPFC,DPSUTOST,DPSTKIDR,DPSAMIFP,DPSCTOFC,DPSTNOFC,DPSTSPFP,DPSOTOFC,DPSTNOFP,DPSTMAFP,DPSTOPFC,DPST11SA,DPSTURNN,DPST06FP,DPST01SA,DPSTEXPA,DPSTHIFP,DPSSTOST,DPSCTOFP,DPSUTOSA,DPSTMSFP,DPST11FP,DPST00ST,DPSTPHFP,DPSATOFC,DPSTHIFC,DPSTBIFC,DPSTTOFP,DPSTURND,DPSTREFP,DPST11FC,DPST00FC,DPSTBAFC,DPSTTWFC,DPST01FP,DPSTMAFC,DPSTFEFP,DPSSTOSA,Year,DPST00FP,DPSTTOST,DPSAMIFC,DPSSTOFP,DPSTINFC,DPSTMSFC,DPSTTOSA,DPSTASFP,DPSTTENA,DPSETOFC,DPSTREFC,DPST06SA,DPSTWHFP,DPSPTOFP,DPSTBLFC,DPSTPIFC,DPSTBIFP,DISTRICT,DPSTCOFP,DPSTVOFP,DPSTCOFC,DPSTTOFC,DPST01FC,DPST06FC,DPSTPIFP,DPST00SA,DPSETOFP,DPSXTOFP,DPSTINFP,DPSTTWFP,DPSTGIFC,DPSTWHFC,DPST11ST
0,0,0,2.3,28.3,235995.1,0,80.5,11.5,38.6,0,5,91696,590847.7,0,64.6,5.3,7.6,7.1,6,433988.8,10.9,11.7,2.1,0.4,11.3,0,0.7,26.8,0,62600,5.6,8.9,46979,14.1,1.8,173585.4,2,57090,18.8,36.5,0,0,107,0.9,0,49.2,48.7,79.2,19.2,0,42.4,0,23.9,14.1,73.2,76964,2025,0,3143733.2,12.5,2.1,0,9.9,59705,0,7.5,14,41.7,50431,92.9,60.4,2.8,0,0,001902,0,9.5,0,52.7,12.6,4.7,0,NaN,13.1,26.5,0,0,0,48.9,1204546.8
1,1.3,0,3.8,38.1,842488.1,0,70.7,22.9,69.2,0,14.2,101314,539939.5,0,101.1,3.2,6.5,3.8,9,458091,13.5,11.3,3,1.9,10.2,0,2.2,21.1,1.1,59941,21.3,19,40314,15.5,3.9,339685,1.8,70158,27.2,35,34150.2,0,170.5,3.4,0,51.4,92.9,72.3,30.7,0.9,62,0,15.3,18.5,78.9,89341,2025,1,5069953.9,19.2,2.2,0,23.8,57787,0,7.9,31.4,63.4,50462,92.9,59.3,2.8,0,0,001903,0,16.2,0,87.7,13.4,16.7,0,39890,18.4,22.3,0,0,0,81.5,1843138.9


In [32]:
# From the matching student and staff dataframes I am combing all of those into one long dataframe, one for student and one for staff
staff_dfs_list = []
student_dfs_list = []

for year, df in staff_matching_col_tables.items():
    staff_dfs_list.append(df)

for year, df in student_matching_col_tables.items():
    student_dfs_list.append(df)

staff_master = pd.concat(staff_dfs_list, ignore_index=True)
student_master = pd.concat(student_dfs_list, ignore_index=True)

In [33]:
student_master.head(2)

,DPETINDP,DPETG01P,DPERRA1R,DPETLEPP,DPETTWOC,DPETRSKP,DPETG06P,DPETECOC,DPETG09C,DPETG09P,DPETG10C,DPERRA4R,DPETG03C,DPETASIC,DPERSA5R,DPETPCIC,DPETG01C,DPETBILC,DPETGPKP,DPETG10P,DPETSPEC,DPERRA6R,DPETG05C,DPETGPKC,DPETRSKC,DPETGEEC,DPETG12P,DPERSA1R,DPETSPEP,DPEMALLC,DPETGIFP,DPETHISC,DPETG07P,DPERRA8R,DPETG12C,DPERRA5R,DPETG11P,DPETG08C,DPETALLC,DPETHISP,DPETG02P,DPETG08P,DPETNEDP,DPETTWOP,DPETBLAC,DPERRAKR,DPEMALLP,DPERSA6R,DPETG04P,DPETBILP,DPETWHIP,DPETG02C,DPERSA4R,DPETASIP,Year,DPETINDC,DPERRA7R,DPETWHIC,DPETNEDC,DPETG06C,DPETECOP,DPERRA3R,DPETG07C,DPERSA7R,DPERSAKR,DPETG05P,DPERSA8R,DPETDISC,DPETGKNP,DPETGEEP,DPETBLAP,DISTRICT,DPETLEPC,DPETG03P,DPETG11C,DPETG04C,DPETPCIP,DPETGKNC,DPETDISP,DPERSA2R,DPERRA2R,DPETGIFC,DPERSA3R
0,0.0,7.1,3.1,0.8,30,27.2,7.2,227,51,8.6,66,0,36,4,0,0,42,5,1.8,11.1,73,0,29,11,162,4,8.4,50,12.3,93,8.9,45,8.4,2.4,50,0,6.6,53,595,7.6,6.6,8.9,61.8,5.0,34,2.9,15.6,0,7.1,0.8,81.0,39,0,0.7,2013,0,0,482,368,43,38.2,10,50,0,0,4.9,0,5,6.7,0.7,5.7,1902,5,6.1,39,42,0.0,40,0.8,0,5.4,53,0
1,0.2,6.5,4.2,1.3,47,27.8,7.4,636,98,7.9,76,0,80,10,0,2,80,16,8.5,6.1,113,7.8,105,105,344,1,6.4,0,9.1,179,4.2,112,7.4,0,79,0,7.8,101,1236,9.1,6.6,8.2,48.5,3.8,54,6.2,14.8,0,7.4,1.3,81.6,82,0,0.8,2013,2,2.2,1009,600,92,51.5,0,91,0,20,8.5,0,5,4.8,0.1,4.4,1903,16,6.5,96,91,0.2,59,0.4,11.1,1.3,52,0


In [34]:
staff_master.head(2)

,DPSTOPFP,DPSECOFC,DPSSTOFC,DPSXTOFC,DPST06ST,DPSTASFC,DPSTBAFP,DPSTURNR,DPSTFEFC,DPSTPHFC,DPSTVOFC,DPSCTOSA,DPST01ST,DPSTGIFP,DPSPTOFC,DPSTBLFP,DPSUTOFC,DPSUTOFP,DPSTSPFC,DPSUTOST,DPSTKIDR,DPSAMIFP,DPSCTOFC,DPSTNOFC,DPSTSPFP,DPSOTOFC,DPSTNOFP,DPSTMAFP,DPSTOPFC,DPST11SA,DPSTURNN,DPST06FP,DPST01SA,DPSTEXPA,DPSTHIFP,DPSSTOST,DPSCTOFP,DPSUTOSA,DPSTMSFP,DPST11FP,DPST00ST,DPSTPHFP,DPSATOFC,DPSTHIFC,DPSTBIFC,DPSTTOFP,DPSTURND,DPSTREFP,DPST11FC,DPST00FC,DPSTBAFC,DPSTTWFC,DPST01FP,DPSTMAFC,DPSTFEFP,DPSSTOSA,Year,DPST00FP,DPSTTOST,DPSAMIFC,DPSSTOFP,DPSTINFC,DPSTMSFC,DPSTTOSA,DPSTASFP,DPSTTENA,DPSETOFC,DPSTREFC,DPST06SA,DPSTWHFP,DPSPTOFP,DPSTBLFC,DPSTPIFC,DPSTBIFP,DISTRICT,DPSTCOFP,DPSTVOFP,DPSTCOFC,DPSTTOFC,DPST01FC,DPST06FC,DPSTPIFP,DPST00SA,DPSETOFP,DPSXTOFP,DPSTINFP,DPSTTWFP,DPSTGIFC,DPSTWHFC,DPST11ST
0,0,22.8,3,28.8,475936,0,87.4,9.8,41.4,0,3.2,76732,232733,0,65.8,3.6,7,6.6,5.4,371739.2,11,13.2,1.6,0,9.8,2,0,23.8,0,46458.6,5.4,22,38785.6,15,5.6,197454,1.4,53006.4,12.6,36.8,30872,0,106,3,0.4,51.2,54.2,83.8,20,1,47.4,0,11,12.8,76.2,65818,2013,1.8,2444371.4,14,2.8,0,6.8,45082,0,7.2,11.6,45.4,40045.4,90.8,62,2,0,0.6,1902,0,5.8,0,54.2,6,11.8,0,30875,10.8,27.2,0,0,0,49.2,929229.6
1,0,0,5,28,752868.2,0,84,19.6,81.6,0,5.4,78268.4,274061.2,2,112.2,1,7.6,5,11.6,347667.8,12.8,4.4,3,2,12,0,2,15.6,0,43717.8,19,21.8,30191,12.6,3.2,300835,2,45384.4,14,34,514659.4,0,155.8,3,0,62,96.4,79.8,33,14.2,81.2,1,9.4,15,84.4,60167,2013,14.6,3915026.2,7,3.2,0,13.4,40536,0,9.4,15.6,77.2,35850,94.8,72,1,0,0,1903,0.6,5.6,0.6,96.6,9,21,0,36340.8,10,18,0,1,2,91.6,1439600
